# Step 1: Import Necessary Libraries

In [6]:
import cv2
import numpy as np
import pygame
import os
import mediapipe as mp
from fer import FER


1. cv2: OpenCV library for image processing and computer vision tasks.
2. numpy: Library for numerical operations.
3. pygame: Library for audio playback.
4. os: Module for interacting with the operating system.
5. mediapipe: Library for detecting hand gestures.
6. fer: Library for facial emotion recognition.

# Step 2: Initialize Pygame Mixer for Audio Control

In [7]:
pygame.mixer.init()

This initializes the Pygame mixer module, which is used for loading and playing music.

# Step 3: Define Functions for Music Control

In [8]:
def play_music(file_path):
    pygame.mixer.music.load(file_path)
    pygame.mixer.music.play()

def pause_music():
    pygame.mixer.music.pause()

def unpause_music():
    pygame.mixer.music.unpause()

def stop_music():
    pygame.mixer.music.stop()

def increase_volume():
    current_volume = pygame.mixer.music.get_volume()
    if current_volume <= 0.9:
        pygame.mixer.music.set_volume(current_volume + 0.1)

def decrease_volume():
    current_volume = pygame.mixer.music.get_volume()
    if current_volume >= 0.1:
        pygame.mixer.music.set_volume(current_volume - 0.1)

def next_song():
    global current_song_index
    current_song_index = (current_song_index + 1) % len(current_playlist)
    play_music(current_playlist[current_song_index])

def previous_song():
    global current_song_index
    current_song_index = (current_song_index - 1) % len(current_playlist)
    play_music(current_playlist[current_song_index])


These functions handle various aspects of music playback, including playing, pausing, stopping, and changing the volume.

# Step 4: Define Folder Paths for Playlists Based on Emotions

In [9]:
playlist_folder = "E:/Song"

This sets the base directory where the music playlists are stored.

# Step 5: Map Emotions to Playlist Folders

In [10]:
emotion_to_playlist = {
    "happy": os.path.join(playlist_folder, "happy"),
    "sad": os.path.join(playlist_folder, "sad")
}

# Step 6: Initialize MediaPipe Hands Model for Gesture Detection

In [11]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils


# Step 7: Initialize FER Emotion Detector

In [12]:
emotion_detector = FER(mtcnn=True)

# Step 8: OpenCV Setup for Emotion Detection

In [14]:
cap = cv2.VideoCapture(0)

# Detect emotion
detected_emotion = None
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally
    frame = cv2.flip(frame, 1)

    # Detect emotions in the frame
    emotion_result = emotion_detector.detect_emotions(frame)
    
    if emotion_result:
        dominant_emotion = max(emotion_result[0]['emotions'], key=emotion_result[0]['emotions'].get)
        if dominant_emotion in ["happy", "sad"]:
            detected_emotion = dominant_emotion
            print(f"Detected Emotion: {detected_emotion}")
            break

    # Display the frame
    cv2.imshow("Emotion Detection", frame)

    # Check for key press events
    key = cv2.waitKey(1)
    if key == ord("q"):
        break

# Release the emotion detection resources
cap.release()
cv2.destroyAllWindows()


Detected Emotion: sad


# Step 9: Load Playlist Based on Detected Emotion

In [15]:
if detected_emotion:
    # Load playlist based on detected emotion
    playlist_path = emotion_to_playlist[detected_emotion]
    current_playlist = [os.path.join(playlist_path, file) for file in os.listdir(playlist_path) if file.endswith(".mp3")]
    current_song_index = 0

    # Play the first song from the playlist
    if current_playlist:
        play_music(current_playlist[current_song_index])


# Step 10: OpenCV Setup for Music Control

In [16]:
    # OpenCV setup for music control
    cap = cv2.VideoCapture(0)

    # Set a larger resolution for the capture (e.g., 1280x720)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    # Define control boxes positions based on the new resolution
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    boxes = {
        "play_pause": (frame_width // 3, frame_height // 3, 2 * frame_width // 3, 2 * frame_height // 3),
        "next": (2 * frame_width // 3 + 20, frame_height // 3, frame_width - 20, 2 * frame_height // 3),
        "previous": (20, frame_height // 3, frame_width // 3 - 20, 2 * frame_height // 3),
        "increase_volume": (frame_width // 3, 20, 2 * frame_width // 3, frame_height // 3 - 20),
        "decrease_volume": (frame_width // 3, 2 * frame_height // 3 + 20, 2 * frame_width // 3, frame_height - 20),
    }

    # Instructions for each control box
    instructions = {
        "play_pause": "Play/Pause",
        "next": "Next Song",
        "previous": "Previous Song",
        "increase_volume": "Increase Volume",
        "decrease_volume": "Decrease Volume"
    }

    # Font properties for instructions
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.8
    font_color = (255, 255, 255)
    line_type = 2

    # Gesture recognition loop
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Flip the frame horizontally
        frame = cv2.flip(frame, 1)

        # Convert the frame to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect hands in the frame
        results = hands.process(rgb_frame)

        # Draw control boxes and instructions
        for key, (x1, y1, x2, y2) in boxes.items():
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            text_size = cv2.getTextSize(instructions[key], font, font_scale, line_type)[0]
            text_x = x1 + (x2 - x1 - text_size[0]) // 2
            text_y = y1 + (y2 - y1 + text_size[1]) // 2
            cv2.putText(frame, instructions[key], (text_x, text_y), font, font_scale, font_color, line_type)

        # Check if hands are detected
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw hand landmarks on the frame
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Get the index finger tip landmark
                index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]

                # Get index finger tip coordinates
                index_finger_x, index_finger_y = int(index_finger_tip.x * frame.shape[1]), int(index_finger_tip.y * frame.shape[0])

                # Check which box the index finger is in
                for key, (x1, y1, x2, y2) in boxes.items():
                    if x1 <= index_finger_x <= x2 and y1 <= index_finger_y <= y2:
                        if key == "play_pause":
                            if pygame.mixer.music.get_busy():
                                pause_music()
                            else:
                                play_music(current_playlist[current_song_index])
                        elif key == "next":
                            next_song()
                        elif key == "previous":
                            previous_song()
                        elif key == "increase_volume":
                            increase_volume()
                        elif key == "decrease_volume":
                            decrease_volume()

        # Display the frame
        cv2.imshow("Gesture-Controlled Music Player", frame)

        # Check for key press events
        key = cv2.waitKey(1)
        if key == ord("q"):
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()
